In [ ]:
import math
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from tensorflow.keras.losses import MeanSquaredError,BinaryCrossentropy,MeanAbsoluteError
import tensorflow as tf
import tensorflow_probability as tfp

In [ ]:
file_path = './train.csv'
file_path2 = './test.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)
df.index = df["id"]
df = pd.DataFrame(df.iloc[:, 1:])
df2 = pd.read_csv(file_path2)
df2.index = df2["id"]
df2 = pd.DataFrame(df2.iloc[:,1:])
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
X_test = df2

# Split the data into training and validating sets
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2,random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, BatchNormalization, Dense
#from tensorflow.keras.initializers import VarianceScaling
from tensorflow.keras.initializers import GlorotUniform

class RegressionModel(tf.keras.Model):
    def __init__(self, input_size):
        super(RegressionModel, self).__init__()

        # Fully connected layers with Xavier initialization
        self.fc1 = layers.Dense(16, input_shape=(input_size,), kernel_initializer='glorot_uniform')
        self.batch_norm1 = layers.BatchNormalization(epsilon=1e-5)
        self.relu1 = layers.Activation('relu')

        self.fc2 = layers.Dense(32, kernel_initializer='glorot_uniform')
        self.batch_norm2 = layers.BatchNormalization(epsilon=1e-5)
        self.gelu = layers.Activation('gelu')

        self.fc3 = layers.Dense(64, kernel_initializer='glorot_uniform')
        self.batch_norm3 = layers.BatchNormalization(epsilon=1e-5)
        self.relu2 = layers.Activation('relu')

        self.fc4 = layers.Dense(1, kernel_initializer='glorot_uniform')

    def call(self, x):
        # Fully connected layers with batch normalization and activation functions
        x = self.relu1(self.batch_norm1(self.fc1(x)))
        x = self.gelu(self.batch_norm2(self.fc2(x)))
        x = self.relu2(self.batch_norm3(self.fc3(x)))

        # Output layer
        x = self.fc4(x)

        return x

# Example usage:
input_size = 11
model = RegressionModel(input_size)

# Optionally, you can build the model to show the summary
model.build((None, input_size))
model.summary()

In [ ]:
def loss_fn(y_true, y_pred):
    return tfp.stats.percentile(tf.abs(y_true - y_pred), q=50)

def metric_fn(y_true, y_pred):
    return tfp.stats.percentile(tf.abs(y_true - y_pred), q=100) - tfp.stats.percentile(tf.abs(y_true - y_pred), q=0)

callbacks_list = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        mode='min',
        patience=30,
        restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, min_lr=0.00001),
]

In [ ]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(0.013, beta_1=0.5), loss=loss_fn, metrics=metric_fn)
model.compile(optimizer=tf.keras.optimizers.Adam(0.013, beta_1=0.5), loss=loss_fn, metrics=metric_fn)

history = model.fit(
    X_train.astype('float32'),
    y_train.astype('float32'),
    epochs=100,
    batch_size=64,
    callbacks=callbacks_list,
    validation_split=0.2
)

In [ ]:
y_pred = model.predict(X_test.astype('float32'))
df_prediction = pd.DataFrame(y_pred)
df_prediction

In [ ]:
from matplotlib import pyplot as plt
df_prediction[0].plot(kind='hist', bins=20, title=0)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
test= pd.read_csv('./sample_submission.csv')
test['Hardness'] = df_prediction
test.to_csv('submission.csv',index=False)

In [ ]:
plt.figure(figsize=(15,4))

plt.subplot(121)
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('Training and validation losses')
plt.legend()

plt.subplot(122)
plt.plot(history.history['metric_fn'], label='Training accuracy')
plt.plot(history.history['val_metric_fn'], label='Validation accuracy')
plt.title('Training and validation accuracies')
plt.legend()

In [ ]:
weights, biases = model.layers[0].get_weights()

# Print or use the final weights as needed
print("가중치:", weights)
print("편향:", biases)